## 1. Setup

In [ ]:
# Install required libraries (run this once if needed)
%pip install numpy pandas matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


## 2. Dataset and Notation

- M: stellar mass (in units of solar mass, M⊙)
- T: effective stellar temperature (Kelvin, K)
- L: stellar luminosity (in units of solar luminosity, L⊙

M = [0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4]

L = [0.15, 0.35, 1.00, 2.30, 4.10, 7.00, 11.2, 17.5, 25.0, 35.0]

In [ ]:
M = np.array([0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4])
L = np.array([0.15, 0.35, 1.00, 2.30, 4.10, 7.00, 11.2, 17.5, 25.0, 35.0])

m = len(M)
l = len(L)

### 2.1 DataSet Visualization: Plot M vs L. 



In [ ]:
plt.figure()
plt.scatter(M, L)
plt.xlabel("Stellar Mass (M☉)")
plt.ylabel("Luminosity (L☉)")
plt.title("Stellar Luminosity vs Mass")
plt.grid(True)
plt.show()


The plot shows a strong positive correlation between stellar mass and luminosity, that is a good sign of some plausibility.

However, the relationship between both is clearly non linear: 
- luminosity increases slowly at low masses and much more rapidly at higher masses.


This suggests that a linear regression model will only provide a rough approximation and will underpredict luminosity for high-mass stars, as this model is not very good for large datasets.